In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from utils_ import *
from mp_models import *
from datasets import *
from gensim import models

In [3]:
train_val_doc_list1_path = '../tmp_files/train_val_doc_list1.pkl'
train_val_doc_list2_path = '../tmp_files/train_val_doc_list2.pkl'
test_doc_list1_path = '../tmp_files/test_doc_list1.pkl'
test_doc_list2_path = '../tmp_files/test_doc_list2.pkl'
train_label_path = "../dataset/newssim-train/label.txt"
test_label_path =  "../dataset/newssim-test/label.txt"
vocab_path = '../tmp_files/vocab.pkl'

In [4]:
with open(train_val_doc_list1_path, 'rb') as f:
    train_val_doc_list1 = pickle.load(f)
with open(train_val_doc_list2_path, 'rb') as f:
    train_val_doc_list2 = pickle.load(f)
with open(test_doc_list1_path, 'rb') as f:
    test_doc_list1 = pickle.load(f)
with open(test_doc_list2_path, 'rb') as f:
    test_doc_list2 = pickle.load(f)
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)

In [5]:
val_data_num = 2000
np.random.seed(3)
indices = np.random.permutation(len(train_val_doc_list1))
train_idx = indices[:-val_data_num]
val_idx = indices[-val_data_num:]

train_doc_list1 = []
train_doc_list2 = []
val_doc_list1 = []
val_doc_list2 = []

for idx in train_idx:
    train_doc_list1.append(train_val_doc_list1[idx])
    train_doc_list2.append(train_val_doc_list2[idx])

for idx in val_idx:
    val_doc_list1.append(train_val_doc_list1[idx])
    val_doc_list2.append(train_val_doc_list2[idx])

In [6]:
print(len(train_doc_list1), len(val_doc_list1), len(test_doc_list1))

25627 2000 2000


In [7]:
train_labels = get_labels(train_label_path)[train_idx]
val_labels = get_labels(train_label_path)[val_idx]
test_labels = get_labels(test_label_path)

In [8]:
print(len(train_labels), len(val_labels), len(test_labels))

25627 2000 2000


In [9]:
max_doc_len = 0
max_sent_len = 0
total_list_len = 0
total_doc_len = 0
total_sent_len = 0
for doc_list in [train_doc_list1, train_doc_list2, val_doc_list1, val_doc_list2]:
    total_list_len += len(doc_list)
    for doc in doc_list:
        max_doc_len = max(max_doc_len, len(doc))
        total_doc_len += len(doc)
        for sent in doc:
            max_sent_len = max(max_sent_len, len(sent))
            total_sent_len += len(sent)
print(f'average document length : {total_doc_len/total_list_len:0.2f}\naverage sentence length : {total_sent_len/total_doc_len:0.2f}')
print(f'max document length : {max_doc_len}\nmax sentence length : {max_sent_len}')

average document length : 24.78
average sentence length : 14.06
max document length : 844
max sentence length : 204


In [10]:
max_document_length = 26 # dummy variable 26
max_sentence_length = 46 # dummy variable 46
first_n_words = None

In [11]:
train_data = NewsDataset(train_doc_list1, train_doc_list2, train_labels, vocab, max_document_length, max_sentence_length, first_n_words)
val_data = NewsDataset(val_doc_list1, val_doc_list2, val_labels, vocab, max_document_length, max_sentence_length, first_n_words)
test_data = NewsDataset(test_doc_list1, test_doc_list2, test_labels, vocab, max_document_length, max_sentence_length, first_n_words)

In [12]:
batch_size = 32

In [13]:
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_data, batch_size=batch_size//4, shuffle=False)

In [14]:
len(vocab)

51053

In [15]:
vocab_size = len(vocab)
embed_dim = 256
pad_idx = vocab('<pad>')
conv_sizes_inner = [[3,3,8], [3,3,16]]
pool_sizes_inner = [[22,22], [5,5]]
hidden_dim_inner = 128
output_dim_inner = 2
conv_sizes_outer = [[3,3,8], [3,3,16]]
pool_sizes_outer = [[22,22], [5,5]]
hidden_dim_outer = 128
learning_rate = 3e-3

In [16]:
'''
vocab_size = len(vocab)
embed_dim = 256
pad_idx = vocab('<pad>')
conv_sizes_inner = [[3,3,8], [3,3,16]]
pool_sizes_inner = [[22,22], [5,5]]
hidden_dim_inner = 128
output_dim_inner = 2
conv_sizes_outer = [[3,3,8], [3,3,16]]
pool_sizes_outer = [[12,12], [5,5]]
hidden_dim_outer = 128
learning_rate = 3e-3
'''

"\nvocab_size = len(vocab)\nembed_dim = 256\npad_idx = vocab('<pad>')\nconv_sizes_inner = [[3,3,8], [3,3,16]]\npool_sizes_inner = [[22,22], [5,5]]\nhidden_dim_inner = 128\noutput_dim_inner = 2\nconv_sizes_outer = [[3,3,8], [3,3,16]]\npool_sizes_outer = [[12,12], [5,5]]\nhidden_dim_outer = 128\nlearning_rate = 3e-3\n"

In [17]:
encoder = DoubleMatchPyramid(vocab_size, embed_dim, pad_idx, conv_sizes_inner, pool_sizes_inner, hidden_dim_inner, output_dim_inner, conv_sizes_outer, pool_sizes_outer, hidden_dim_outer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    encoder = nn.DataParallel(encoder)
encoder.to(device)

DataParallel(
  (module): DoubleMatchPyramid(
    (embed): Embedding(51053, 256, padding_idx=0)
    (mp_inner): MatchPyramid(
      (conv): ModuleList(
        (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
        (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
      )
      (bn): ModuleList(
        (0): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): ModuleList(
        (0): AdaptiveAvgPool2d(output_size=(22, 22))
        (1): AdaptiveAvgPool2d(output_size=(5, 5))
      )
      (linear1): Linear(in_features=400, out_features=128, bias=True)
      (linear2): Linear(in_features=128, out_features=2, bias=True)
    )
    (conv): ModuleList(
      (0): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1))
      (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    )
    (bn): ModuleList(
      (0): BatchNorm2d(8, eps=1e-05, momentum=0

In [19]:
min_loss = 10000
max_acc = 0

In [20]:
num_epoches = 5
train_batch_num = len(train_data_loader)
val_batch_num = len(val_data_loader)
log_step = 80
start_time = time.time()

for epoch in range(num_epoches):

    print("{:-^50s}".format("TRAIN"))
    encoder.train()
    total_loss = 0
    for i, (t1, t2, target) in enumerate(train_data_loader):

        t1 = t1.to(device)
        t2 = t2.to(device)
        target = target.long().to(device)
        
        out = encoder(t1, t2)
        loss = criterion(out, target)
        encoder.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        if (i+1) % log_step == 0:
            print(f'epoch: {epoch+1:2d}/{num_epoches:2d}, batch: {i+1:3d}/{train_batch_num}, elapsed_time: {time.time()-start_time:10.4f}s')

    print(f'epoch: {epoch+1:2d}/{num_epoches:2d}, average_loss_per_batch: {total_loss/train_batch_num:5.4f}\n')

    print("{:-^50s}".format("VALIDATION"))
    encoder.eval()
    total_loss = 0
    with torch.no_grad():
        out_list = []
        target_list = []
        for i, (t1, t2, target) in enumerate(val_data_loader):

            t1 = t1.to(device)
            t2 = t2.to(device)
            target = target.long().to(device)
            
            out = encoder(t1, t2)
            loss = criterion(out, target)
            total_loss += loss.item()

            out_list.append(out.detach().cpu().numpy())
            target_list.append(target.detach().cpu().numpy())

            if (i+1) % (log_step//4) == 0:
                print(f'epoch: {epoch+1:2d}/{num_epoches:2d}, batch: {i+1:3d}/{val_batch_num}, elapsed_time: {time.time()-start_time:10.4f}s')

        out = np.concatenate(out_list, axis=0).argmax(1)
        target = np.concatenate(target_list, axis=0)
        acc = np.sum(out == target) / len(out)
        print(f'accuracy : {acc:.4f}')
        
        print(f'epoch: {epoch+1:2d}/{num_epoches:2d}, average_loss_per_batch: {total_loss/val_batch_num:5.4f}\n')

        if total_loss/val_batch_num < min_loss:
            min_loss = total_loss/val_batch_num
            torch.save(encoder.state_dict(), './parameters/dmp_loss.pt')
            print("model parameters saved : loss\n")

        if acc > max_acc:
            max_acc = acc
            torch.save(encoder.state_dict(), './parameters/dmp_acc.pt')
            print("model parameters saved : acc\n")

----------------------TRAIN-----------------------
epoch:  1/ 5, batch:  80/801, elapsed_time:    19.8906s
epoch:  1/ 5, batch: 160/801, elapsed_time:    36.5904s
epoch:  1/ 5, batch: 240/801, elapsed_time:    53.1877s
epoch:  1/ 5, batch: 320/801, elapsed_time:    71.4610s
epoch:  1/ 5, batch: 400/801, elapsed_time:    87.6430s
epoch:  1/ 5, batch: 480/801, elapsed_time:   105.0895s
epoch:  1/ 5, batch: 560/801, elapsed_time:   121.1743s
epoch:  1/ 5, batch: 640/801, elapsed_time:   138.2565s
epoch:  1/ 5, batch: 720/801, elapsed_time:   155.2982s
epoch:  1/ 5, batch: 800/801, elapsed_time:   171.9716s
epoch:  1/ 5, average_loss_per_batch: 0.4214

--------------------VALIDATION--------------------
epoch:  1/ 5, batch:  20/63, elapsed_time:   174.2306s
epoch:  1/ 5, batch:  40/63, elapsed_time:   176.3227s
epoch:  1/ 5, batch:  60/63, elapsed_time:   178.7628s
accuracy : 0.8520
epoch:  1/ 5, average_loss_per_batch: 0.3459

model parameters saved : loss

model parameters saved : acc

--

In [41]:
# encoder.load_state_dict(torch.load('./parameters/dmp_loss.pt'))
# encoder.load_state_dict(torch.load('./parameters/dmp_acc.pt'))
encoder.load_state_dict(torch.load('./parameters/dmp_best.pt'))

<All keys matched successfully>

In [42]:
out_list_test = []
target_list_test = []
encoder.eval()

for i, (t1, t2, target) in enumerate(test_data_loader):
    
    t1 = t1.to(device)
    t2 = t2.to(device)
    target = target.long().to(device)
    
    out = encoder(t1, t2)
    
    out_list_test.append(out.detach().cpu().numpy().argmax(1))
    target_list_test.append(target.detach().cpu().numpy())

In [43]:
out_test = np.concatenate(out_list_test, axis=0)
target_test = np.concatenate(target_list_test, axis=0)
predictions = out_test
labels = target_test

In [44]:
print_test_statistics(predictions, labels)

accuracy : 0.8335
precision : 0.7886
recall : 0.7753
F1 score : 0.7819
TP :  597  FN :  173
FP :  160  TN : 1070
